# Reshaping Data with pandas

Data formats include long format, wide format, and mixed format.

* Long data format is based on 3 parts: Index, Attributes, and Values. The attributes are usually stored in a single column. However, for hierarchical columns from wide format, nested columns are present in diffent attribute column. Long data is used for calculations.

* Wide format arranges data such that each entity has one row, with every attribute value spread across single row. If the data is hierarchical, the columns are also hierarchical for example same attr for following years. In this format, each attribute is represented in a separate column for each entity. NaN values are common in wide format, as not every entity will have a value for every attribute. Wide data is used for compare each entity's attributes. Transpose could be used to compare each attributes between entities.

* Mixed format combines both long and wide formats, typically uses a MultiIndex. It can be considered an unperfect version of both long and wide formats. Mixed format (multiindex and hierarchical columns) could be usefull but for sure allows to see the data from diffrent perspective.

To move around between long-wide-transposed-mixed we use following methods : 
* pd.melt()
* DataFrame.transpose()
* DataFrame.pivot()
* DataFrame.pivot_table()
* pd.wide_to_long()


In [3]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=Warning)

In [4]:
data_frame = pd.read_csv("../PROJECTS/HOUSING/data/sales/2007-property-sales-data.csv")

In [5]:
data_frame.head(2)

,PropType,Taxkey,Address,CondoProject,District,Nbhd,Style,Extwall,Stories,Year_Built,Nr_of_rms,Fin_sqft,Units,Bdrms,Fbath,Hbath,Lotsize,Sale_date,Sale_price
0,Lg Apartment,2459989000,2330 W ROOSEVELT DR,51: 1-Br,1,6975,Apartment: 51: 1-Br Units,NaN,3.0,1929,0,44652,1,0,0,0,0,2007-01,880000
1,Lg Apartment,3590295000,1626 N PROSPECT AV,"79: 1-Br ,40: 1-Br ,43: 2-Br",4,6974,"Apartment: 79: 1-Br ,40: 1-Br ,43: 2-Br Units",NaN,22.0,1964,0,230828,1,0,0,0,38010,2007-01,22500000


In [6]:
# lets make address as entity that corresponds to one sale

In [7]:
data_frame["Address"].value_counts().sort_values(ascending=False).head(10)

Address
270 E HIGHLAND AV        5
9296 N BURBANK AV        2
9306 N BURBANK AV        2
4732 W FOND DU LAC AV    2
1314 W NATIONAL AV       2
1559 W LINCOLN AV        1
3733 W NATIONAL AV       1
1100 E GARFIELD AV       1
3164 S 13TH ST           1
4433 W LISBON AV         1
Name: count, dtype: int64

In [8]:
data_frame.shape

(283, 19)

In [9]:
# we can see that some of the addresses are not unique lets drop the rows

In [10]:
data_frame.drop_duplicates(subset="Address", inplace=True)

In [11]:
data_frame.shape

(275, 19)

In [12]:
data_frame["Address"].value_counts().sort_values(ascending=False).head(10)

Address
2330 W ROOSEVELT DR    1
1559 W LINCOLN AV      1
3733 W NATIONAL AV     1
1100 E GARFIELD AV     1
3164 S 13TH ST         1
4433 W LISBON AV       1
4122 W CAPITOL DR      1
2912 S 13TH ST         1
3720 W VLIET ST        1
9520 N 107TH ST        1
Name: count, dtype: int64

In [13]:
# lets subset only some of the columns

In [14]:
data_frame = data_frame.loc[ : , ["PropType", "Address", "District", "Year_Built", "Fin_sqft", "Bdrms", "Fbath", "Sale_date", "Sale_price"] ]

In [15]:
data_frame_wide_idx = data_frame.set_index("Address")

In [16]:
# wide dataframe has every attribute value in one row also one row is for each entity - single separate sale
# our columns are not nested which is common

In [17]:
data_frame_wide_idx.head()

,PropType,District,Year_Built,Fin_sqft,Bdrms,Fbath,Sale_date,Sale_price
Address,,,,,,,,
2330 W ROOSEVELT DR,Lg Apartment,1,1929,44652,0,0,2007-01,880000
1626 N PROSPECT AV,Lg Apartment,4,1964,230828,0,0,2007-01,22500000
3030 W PIERCE ST,Lg Apartment,8,1928,7940,0,0,2007-01,720000
3916 W KISSLICH PL,Commercial,15,1909,8272,0,0,2007-01,42000
4768 N HOPKINS ST,Commercial,7,1925,2610,0,0,2007-01,64000


In [18]:
# we can subset our wide dataframe by using loc[] and passing index concrete values and columns labels

In [19]:
data_frame_wide_idx.loc["3030 W PIERCE ST" : "4768 N HOPKINS ST", ["PropType", "Sale_date"]]

,PropType,Sale_date
Address,,
3030 W PIERCE ST,Lg Apartment,2007-01
3916 W KISSLICH PL,Commercial,2007-01
4768 N HOPKINS ST,Commercial,2007-01


In [20]:
# wide data frame allows us to easily read data of individal entity, if we want to compare specific attribute for many entitty
# we will use transpose to swap columns with rows

In [21]:
data_frame_wide_idx.transpose().iloc[ :5,  :5 ]

Address,2330 W ROOSEVELT DR,1626 N PROSPECT AV,3030 W PIERCE ST,3916 W KISSLICH PL,4768 N HOPKINS ST
PropType,Lg Apartment,Lg Apartment,Lg Apartment,Commercial,Commercial
District,1,4,8,15,7
Year_Built,1929,1964,1928,1909,1925
Fin_sqft,44652,230828,7940,8272,2610
Bdrms,0,0,0,0,0


In [22]:
# now lets convert wide format into long format that will suit to machine learning models
# below dataframe contains 1 level of columns and unique identity for each row

In [23]:
data_frame_wide_idx.head(4)

,PropType,District,Year_Built,Fin_sqft,Bdrms,Fbath,Sale_date,Sale_price
Address,,,,,,,,
2330 W ROOSEVELT DR,Lg Apartment,1,1929,44652,0,0,2007-01,880000
1626 N PROSPECT AV,Lg Apartment,4,1964,230828,0,0,2007-01,22500000
3030 W PIERCE ST,Lg Apartment,8,1928,7940,0,0,2007-01,720000
3916 W KISSLICH PL,Commercial,15,1909,8272,0,0,2007-01,42000


In [24]:
# pd.melt?
data_frame_wide = data_frame_wide_idx.reset_index(drop = False)

In [25]:
data_frame_wide

,Address,PropType,District,Year_Built,Fin_sqft,Bdrms,Fbath,Sale_date,Sale_price
0,2330 W ROOSEVELT DR,Lg Apartment,1,1929,44652,0,0,2007-01,880000
1,1626 N PROSPECT AV,Lg Apartment,4,1964,230828,0,0,2007-01,22500000
2,3030 W PIERCE ST,Lg Apartment,8,1928,7940,0,0,2007-01,720000
3,3916 W KISSLICH PL,Commercial,15,1909,8272,0,0,2007-01,42000
4,4768 N HOPKINS ST,Commercial,7,1925,2610,0,0,2007-01,64000
...,...,...,...,...,...,...,...,...,...
270,7625 W FOND DU LAC AV,Commercial,2,1996,15000,0,0,2007-12,2150000
271,3800 W BURNHAM ST,Commercial,8,1910,4270,0,0,2007-12,300000
272,3261 S 13TH ST,Commercial,14,1910,2210,0,0,2007-12,100000
273,3304 W PIERCE ST,Commercial,8,1949,6240,0,0,2007-12,250000


In [26]:
data_frame_long = pd.melt(frame=data_frame_wide, id_vars = "Address", var_name = "attr", value_name="value")
data_frame_long_srt = data_frame_long.sort_values("Address")
data_frame_long_srt_idx = data_frame_long_srt.set_index("Address")

In [27]:
# below subset for specific address with all atributes and corresponding values, follows the rule one observation for one row
data_frame_long_srt_idx.loc["10000 W CAPITOL DR", : ]

,attr,value
Address,,
10000 W CAPITOL DR,Year_Built,1962
10000 W CAPITOL DR,Bdrms,0
10000 W CAPITOL DR,Sale_price,290000
10000 W CAPITOL DR,PropType,Commercial
10000 W CAPITOL DR,Fin_sqft,1952
10000 W CAPITOL DR,Fbath,0
10000 W CAPITOL DR,District,5
10000 W CAPITOL DR,Sale_date,2007-08


In [28]:
# importing and concat to create wide dataframe with hierarchical columns
# our example will be fifa players, fifa game is released in every year, we will read data for 3 years then find players present in all 3 editions
# then we will create small dataframe with thease duplicates to show how wide data with nested columns looks

In [29]:
fifa_22 = pd.read_csv("../PROJECTS/FIFA/players_22.csv")
fifa_21 = pd.read_csv("../PROJECTS/FIFA/players_21.csv")
fifa_20 = pd.read_csv("../PROJECTS/FIFA/players_20.csv")

In [30]:
fifa_22.head(2)

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,...,50+3,50+3,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,...,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png


In [31]:
fifa_21.head(2)

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,103500000.0,560000.0,33,...,52+3,52+3,52+3,62+3,19+3,https://cdn.sofifa.net/players/158/023/21_120.png,https://cdn.sofifa.net/teams/241/60.png,https://cdn.sofifa.net/flags/es.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",92,92,63000000.0,220000.0,35,...,54+3,54+3,54+3,61+3,20+3,https://cdn.sofifa.net/players/020/801/21_120.png,https://cdn.sofifa.net/teams/45/60.png,https://cdn.sofifa.net/flags/it.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png


In [32]:
# lets choose some attributes
list(fifa_20.columns)

['sofifa_id',
 'player_url',
 'short_name',
 'long_name',
 'player_positions',
 'overall',
 'potential',
 'value_eur',
 'wage_eur',
 'age',
 'dob',
 'height_cm',
 'weight_kg',
 'club_team_id',
 'club_name',
 'league_name',
 'league_level',
 'club_position',
 'club_jersey_number',
 'club_loaned_from',
 'club_joined',
 'club_contract_valid_until',
 'nationality_id',
 'nationality_name',
 'nation_team_id',
 'nation_position',
 'nation_jersey_number',
 'preferred_foot',
 'weak_foot',
 'skill_moves',
 'international_reputation',
 'work_rate',
 'body_type',
 'real_face',
 'release_clause_eur',
 'player_tags',
 'player_traits',
 'pace',
 'shooting',
 'passing',
 'dribbling',
 'defending',
 'physic',
 'attacking_crossing',
 'attacking_finishing',
 'attacking_heading_accuracy',
 'attacking_short_passing',
 'attacking_volleys',
 'skill_dribbling',
 'skill_curve',
 'skill_fk_accuracy',
 'skill_long_passing',
 'skill_ball_control',
 'movement_acceleration',
 'movement_sprint_speed',
 'movement_agi

In [33]:
fifa_20.head(2)

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, CF, ST",94,94,95500000.0,560000.0,32,...,53+6,53+6,53+6,63+6,19+6,https://cdn.sofifa.net/players/158/023/20_120.png,https://cdn.sofifa.net/teams/241/60.png,https://cdn.sofifa.net/flags/es.png,NaN,https://cdn.sofifa.net/flags/ar.png
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",93,93,58500000.0,410000.0,34,...,54+3,54+3,54+3,61+3,20+3,https://cdn.sofifa.net/players/020/801/20_120.png,https://cdn.sofifa.net/teams/45/60.png,https://cdn.sofifa.net/flags/it.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png


In [34]:
attributes_names = ["club_name", "overall", "value_eur", "short_name"]
fifa_20 = fifa_20.loc[ : , attributes_names]
fifa_21 = fifa_21.loc[ : , attributes_names]
fifa_22 = fifa_22.loc[ : , attributes_names]

In [35]:
fifa_20.head(2)

,club_name,overall,value_eur,short_name
0,FC Barcelona,94,95500000.0,L. Messi
1,Juventus,93,58500000.0,Cristiano Ronaldo


In [36]:
fifa_21.head(2)

,club_name,overall,value_eur,short_name
0,FC Barcelona,93,103500000.0,L. Messi
1,Juventus,92,63000000.0,Cristiano Ronaldo


In [37]:
fifa_22.head(2)

,club_name,overall,value_eur,short_name
0,Paris Saint-Germain,93,78000000.0,L. Messi
1,FC Bayern München,92,119500000.0,R. Lewandowski


In [74]:
# pd.wide_to_long() allows convert wide format with suffix columns labels

In [76]:
df_suffix = pd.DataFrame({
    "author" : ["Dan Brown", "Chis Isak"],
    "publisher_code" : [12, 10],
    "publisher_name" : ["publiser1", "publisher2"],
    "language_code" : [12, 10],
    "language_code" : ["english", "spanish"]
})

In [78]:
df_suffix

,author,publisher_code,publisher_name,language_code
0,Dan Brown,12,publiser1,english
1,Chis Isak,10,publisher2,spanish


In [82]:
# set title ti index, the sep is underscore "_". stubnames corresponds to attributes in new dataframe, the part after suffix
# will be added in new column named with parameter "j". Suffix by default expect numerical values after sep

In [90]:
pd.wide_to_long(
    df_suffix,
    i = "author",
    j = "type",
    stubnames = ["language", "publisher"],
    sep = "_",
    suffix="\w+"
)

,,language,publisher
author,type,,
Dan Brown,code,english,12
Chis Isak,code,spanish,10
Dan Brown,name,NaN,publiser1
Chis Isak,name,NaN,publisher2
